## Calling the DOMO instance Extract the key datasets created

In [ ]:
from pydomo import Domo

import pandas as pd

client_id = 'Client_id'
client_secret = 'Client_Secret'

domo = Domo(client_id,client_secret,api_host='api.domo.com')
df = domo.ds_get('dataset_id URL')
df.head()


dataset_id = df['DATASET_IDS_NAME']
print(dataset_id)
dataset_name = df['DATASET_NAME']
print(dataset_name)

## Creating a Session Token

In [ ]:

import requests
import json
domo_instance = 'DOMO INSTANCE'

url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

payload = {
    "method": "password",
    "emailAddress": "',
    "password": ""
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, data=json.dumps(payload), headers=headers)

if response.status_code == 200:
    response_data = response.json()
    session_token = response_data.get('sessionToken', '')
    print(session_token)
else:
    print("Request failed with status code:", response.status_code)


## Capturing the Stream_id using the dataset_id

In [ ]:


domo_instance = 'DOMO INSTANCE'


session = requests.Session()
session.headers.update({
    "x-domo-authentication": session_token,
    "Content-Type": "application/json"
})

stream_ids = []  
dataset_ids = []

for id in dataset_id:
    url = f"https://{domo_instance}.domo.com/api/data/v3/datasources/{id}"

    try:
        response = session.get(url)  
        response.raise_for_status() 
        response_data = response.json()
        

        stream_id = response_data.get('streamId')  
        dataset_id = response_data.get('id') 

        if stream_id:
            stream_ids.append(stream_id)  
            print("Stream ID:", stream_id)
        else:
            print("Stream ID not found for Dataset ID:", id)

        if  dataset_id:
            dataset_ids.append(dataset_id)  
            print("Dataset ID:", dataset_id)
        else:
            print("Dataset ID not found for Dataset ID:", id)


        

    except requests.exceptions.RequestException as e:
        print(f"An error occurred for Dataset ID: {id}")
        print(f"Error details: {e}")


print("Stream IDs:", stream_ids)
print( "Dataset_ids:",dataset_ids)


## Generation the configuraion body and capturing it the Output

In [ ]:
import re
import time
import pandas as pd
import requests


domain = "DOMO DOMAIN"
access_token = "ACCESS TOKEN"

session = requests.Session()
session.headers.update({
    "X-Domo-Developer-Token": access_token,
    "Content-Type": "application/json"
})

delay = 0.1

stream_data = []

for streamid in stream_ids:
    url = f"https://{domain}/api/data/v1/streams/{streamid}"

    try:
        response = session.get(url)
        response.raise_for_status()
        response_data = response.json()

        json_data = response_data

        if isinstance(json_data, list):
            if json_data:
                for item in json_data:
                    data_source = item["configuration"]
                    output_text = str(data_source)  
                    stream_data.append((streamid, output_text,dataset_name))
            else:
                print("No data available in the response.")
        else:
            data_source = json_data["configuration"]
            output_text = str(data_source)  
            stream_data.append((streamid, output_text,dataset_name))

    except requests.exceptions.RequestException as e:
        print(f"An error occurred for Stream ID: {streamid}")
        print(f"Error details: {e}")

    time.sleep(delay)

# Create a DataFrame from the captured stream data
df = pd.DataFrame(stream_data, columns=["Stream ID", "Configuration","Dataset Name"])
print(df)




## Post request taking the configuration Body and creating a new connector in the Domo Instance 

In [ ]:
import json
import ast  

domain = "DOMO INSTANCE"
access_token = "ACCESS TOKEN"


session = requests.Session()
session.headers.update({
    "X-Domo-Developer-Token": access_token,
    "Content-Type": "application/json"
})

delay = 3

url = "https://{DOMO INSTANCE}.domo.com/api/data/v1/streams"

for index, row in df.iterrows():
    datasource = row["Configuration"]
    datasetname = row["Stream ID"]

    payload = {
        "transport": {
            "type": "CONNECTOR",
            "description": "com.domo.connector.adobeanalyticsv2",
            "version": 3
        },
        "configuration": ast.literal_eval(datasource),  
        "account": {
            "id": 103
        },
        "updateMethod": "REPLACE",
        "dataProvider": {
            "key": "adobe-analytics-oauth"
        },
        "dataSource": {
            "name": datasetname,
            "description": "",
            "cloudId": "LOCATION ID "
        },
        "advancedScheduleJson": "{\"type\":\"DAY\",\"at\":\"05:27 AM\",\"timezone\":\"\"}",
        "scheduleRetryExpression": None,
        "scheduleRetryCount": 0
    }
    headers = {
        'authority': 'rolex.domo.com',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en',
        'content-type': 'application/json;charset=utf-8',
        'origin': 'https://rolex.domo.com',
        'x-domo-requestcontext': '{"clientToe":""}',
        'x-requested-with': 'XMLHttpRequest'
    }

    response = session.request("POST", url, headers=headers, json=payload)

    print(response.json())

    time.sleep(delay)


## Mapping of Actual Dataset Name


In [ ]:
from pydomo import Domo

import pandas as pd

client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'

domo = Domo(client_id,client_secret,api_host='api.domo.com')
df = domo.ds_get('Dataset_url_id')
df.head()


dataset_dataset_id_NEW = df['Dataset ID']
print(dataset_dataset_id_NEW)
dataset_name_NEW = df['Dataset_Name']
print(dataset_name_NEW)

## Generating a Bearer Token

In [ ]:
import requests

url = "https://api.domo.com/oauth/token?grant_type=client_credentials"

payload = {
    'username': 'CLIENT_ID',
    'password': 'CLIENT_SECRET'
}

headers = {
    'Authorization': 'POST IN POSTMAN'
}

response_new = requests.request("GET", url, headers=headers, data=payload)

if response_new.status_code == 200:
    response_data_new = response_new.json()
    bearer_token = response_data_new.get('access_token','')
    print(bearer_token)
else:
    print("Request failed with status code:", response.status_code)





## Creating the mapping into the Domo Instance

In [ ]:
import requests
import time

domo_instance = "DOMO INSTANCE"


session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    'Authorization': f'Bearer {bearer_token}',
    
})
delay = 0.01

for dataset_id, table_name in zip(dataset_dataset_id_NEW, dataset_name_NEW):
    url = f"https://api.domo.com/v1/datasets/{dataset_id}"  

    payload = {
        "name": table_name,
        "standardize": True,
    }

    response = session.put(url, headers=session.headers, json=payload)
    response.raise_for_status()  

    print(f"Mapping updated successfully for table '{table_name}' in dataset ID '{dataset_id}'")

    print(response)

    
    time.sleep(delay)
